In [1]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mutual_info_score, roc_curve, auc, confusion_matrix, classification_report, roc_auc_score, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
from mrmr import mrmr_classif
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
from scipy.stats import norm

In [2]:
### One example of dual-modality PET-CT stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# Read data (CT)
train_data_CT = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_CT_bin50\6-4ADASYN-11\train.csv')
test_data_CT = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_CT_bin50\6-4ADASYN-11\test.csv')

X_train_CT = train_data_CT.iloc[:, 1:]  
y_train_CT = train_data_CT.iloc[:, 0]   
X_test_CT = test_data_CT.iloc[:, 1:]  
y_test_CT = test_data_CT.iloc[:, 0]

In [3]:
### One example of dual-modality Combined stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# Read data (PET)
train_data_PET = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-11\original\train.csv')
test_data_PET = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-11\original\test.csv')

X_train_PET = train_data_PET.iloc[:, 1:]  
y_train_PET = train_data_PET.iloc[:, 0]   
X_test_PET = test_data_PET.iloc[:, 1:]   
y_test_PET = test_data_PET.iloc[:, 0]    

In [4]:
### One example of dual-modality PET-CT stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
#Classifier (CT) --- Random Forest
np.random.seed(0)
param_grid_CT = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],    
    'bootstrap': [True, False]        
}
rf = RandomForestClassifier()
grid_search_CT = GridSearchCV(estimator=rf, param_grid=param_grid_CT, cv=5, scoring='accuracy')
grid_search_CT.fit(X_train_CT, y_train_CT)
print("Best parameters:", grid_search_CT.best_params_)
best_model_CT = grid_search_CT.best_estimator_

y_train_pred_cv_CT = cross_val_predict(best_model_CT, X_train_CT, y_train_CT, cv=5)
y_train_pred_proba_cv_CT = cross_val_predict(best_model_CT, X_train_CT, y_train_CT, cv=5, method='predict_proba')[:, 1]
train_precision_cv_CT, train_recall_cv_CT, train_f1_cv_CT, _ = precision_recall_fscore_support(y_train_CT, y_train_pred_cv_CT, average='binary')
y_test_pred_CT = best_model_CT.predict(X_test_CT)
y_test_pred_proba_CT = best_model_CT.predict_proba(X_test_CT)[:, 1]
test_precision_CT, test_recall_CT, test_f1_CT, _ = precision_recall_fscore_support(y_test_CT, y_test_pred_CT, average='binary')

train_accuracy_cv_CT = cross_val_score(best_model_CT, X_train_CT, y_train_CT, cv=5, scoring='accuracy').mean()
test_accuracy_CT = accuracy_score(y_test_CT, y_test_pred_CT)
conf_mat_train_cv_CT = confusion_matrix(y_train_CT, y_train_pred_cv_CT)
conf_mat_test_CT = confusion_matrix(y_test_CT, y_test_pred_CT)
sensitivity_train_CT = conf_mat_train_cv_CT[1, 1] / (conf_mat_train_cv_CT[1, 1] + conf_mat_train_cv_CT[1, 0])  # TP / (TP + FN)
sensitivity_test_CT = conf_mat_test_CT[1, 1] / (conf_mat_test_CT[1, 1] + conf_mat_test_CT[1, 0])  # TP / (TP + FN)
specificity_train_CT = conf_mat_train_cv_CT[0, 0] / (conf_mat_train_cv_CT[0, 0] + conf_mat_train_cv_CT[0, 1])
specificity_test_CT = conf_mat_test_CT[0, 0] / (conf_mat_test_CT[0, 0] + conf_mat_test_CT[0, 1])
ppv_train_CT = conf_mat_train_cv_CT[1, 1] / (conf_mat_train_cv_CT[1, 1] + conf_mat_train_cv_CT[0, 1])
npv_train_CT = conf_mat_train_cv_CT[0, 0] / (conf_mat_train_cv_CT[0, 0] + conf_mat_train_cv_CT[1, 0])
ppv_test_CT = conf_mat_test_CT[1, 1] / (conf_mat_test_CT[1, 1] + conf_mat_test_CT[0, 1])
npv_test_CT = conf_mat_test_CT[0, 0] / (conf_mat_test_CT[0, 0] + conf_mat_test_CT[1, 0])
train_precision_CT = precision_score(y_train_CT, y_train_pred_cv_CT)
test_precision_CT = precision_score(y_test_CT, y_test_pred_CT)
train_recall_CT = recall_score(y_train_CT, y_train_pred_cv_CT)
test_recall_CT = recall_score(y_test_CT, y_test_pred_CT)
train_f1_CT = f1_score(y_train_CT, y_train_pred_cv_CT)
test_f1_CT = f1_score(y_test_CT, y_test_pred_CT)
print(f"Sensitivity (Cross-Validation_CT): {sensitivity_train_CT:.4f}")
print(f"Specificity (Cross-Validation_CT): {specificity_train_CT:.4f}")
print(f"PPV (Cross-Validation_CT): {ppv_train_CT:.4f}")
print(f"NPV (Cross-Validation_CT): {npv_train_CT:.4f}")
print(f"Precision (Cross-Validation_CT): {train_precision_CT:.4f}")
print(f"Recall (Cross-Validation_CT): {train_recall_CT:.4f}")
print(f"F1 Score (Cross-Validation_CT): {train_f1_CT:.4f}")
print(f"Train Accuracy (Cross-Validation_CT): {train_accuracy_cv_CT:.4f}")
print(f"Sensitivity (Test_CT): {sensitivity_test_CT:.4f}")
print(f"Specificity (Test_CT): {specificity_test_CT:.4f}")
print(f"PPV (Test_CT): {ppv_test_CT:.4f}")
print(f"NPV (Test_CT): {npv_test_CT:.4f}")
print(f"Precision (Test_CT): {test_precision_CT:.4f}")
print(f"Recall (Test_CT): {test_recall_CT:.4f}")
print(f"F1 Score (Test_CT): {test_f1_CT:.4f}")
print(f"Test Accuracy_CT: {test_accuracy_CT:.4f}")

fpr_train_CT, tpr_train_CT, _ = roc_curve(y_train_CT, y_train_pred_proba_cv_CT)
fpr_test_CT, tpr_test_CT, _ = roc_curve(y_test_CT, y_test_pred_proba_CT)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train_CT = bootstrap_roc_auc(y_train_CT, y_train_pred_proba_cv_CT)
auc_train_mean_CT = np.mean(auc_scores_train_CT)
auc_train_std_CT = np.std(auc_scores_train_CT)
auc_train_lower_CT = norm.ppf(0.025, loc=auc_train_mean_CT, scale=auc_train_std_CT)
auc_train_upper_CT = norm.ppf(0.975, loc=auc_train_mean_CT, scale=auc_train_std_CT)
print(f"Cross-Validation AUC_CT: {auc_train_mean_CT:.4f} (95% CI: {auc_train_lower_CT:.4f}, {auc_train_upper_CT:.4f})")

auc_scores_test_CT = bootstrap_roc_auc(y_test_CT, y_test_pred_proba_CT)
auc_test_mean_CT = np.mean(auc_scores_test_CT)
auc_test_std_CT = np.std(auc_scores_test_CT)
auc_test_lower_CT = norm.ppf(0.025, loc=auc_test_mean_CT, scale=auc_test_std_CT)
auc_test_upper_CT = norm.ppf(0.975, loc=auc_test_mean_CT, scale=auc_test_std_CT)
print(f"Test AUC_CT: {auc_test_mean_CT:.4f} (95% CI: {auc_test_lower_CT:.4f}, {auc_test_upper_CT:.4f})")

Best parameters: {'bootstrap': False, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Sensitivity (Cross-Validation_CT): 0.8438
Specificity (Cross-Validation_CT): 0.7255
PPV (Cross-Validation_CT): 0.7431
NPV (Cross-Validation_CT): 0.8315
Precision (Cross-Validation_CT): 0.7431
Recall (Cross-Validation_CT): 0.8438
F1 Score (Cross-Validation_CT): 0.7902
Train Accuracy (Cross-Validation_CT): 0.7783
Sensitivity (Test_CT): 0.5667
Specificity (Test_CT): 0.8625
PPV (Test_CT): 0.6071
NPV (Test_CT): 0.8415
Precision (Test_CT): 0.6071
Recall (Test_CT): 0.5667
F1 Score (Test_CT): 0.5862
Test Accuracy_CT: 0.7818
Cross-Validation AUC_CT: 0.8586 (95% CI: 0.8070, 0.9102)
Test AUC_CT: 0.8266 (95% CI: 0.7457, 0.9075)


In [5]:
### One example of dual-modality PET-CT stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
#Classifier (PET) --- Logistic Regression
np.random.seed(0)
param_grid_PET = {
    'C': [0.1, 1.0, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'max_iter': [1000],
    'tol': [1e-4]
}
logistic = LogisticRegression()
grid_search_PET = GridSearchCV(estimator=logistic, param_grid=param_grid_PET, cv=5, scoring='accuracy')
grid_search_PET.fit(X_train_PET, y_train_PET)
print("Best parameters:", grid_search_PET.best_params_)
best_model_PET = grid_search_PET.best_estimator_

y_train_pred_cv_PET = cross_val_predict(best_model_PET, X_train_PET, y_train_PET, cv=5)
y_train_pred_proba_cv_PET = cross_val_predict(best_model_PET, X_train_PET, y_train_PET, cv=5, method='predict_proba')[:, 1]
train_precision_cv_PET, train_recall_cv_PET, train_f1_cv_PET, _ = precision_recall_fscore_support(y_train_PET, y_train_pred_cv_PET, average='binary')
y_test_pred_PET = best_model_PET.predict(X_test_PET)
y_test_pred_proba_PET = best_model_PET.predict_proba(X_test_PET)[:, 1]
test_precision_PET, test_recall_PET, test_f1_PET, _ = precision_recall_fscore_support(y_test_PET, y_test_pred_PET, average='binary')

train_accuracy_cv_PET = cross_val_score(best_model_PET, X_train_PET, y_train_PET, cv=5, scoring='accuracy').mean()
test_accuracy_PET = accuracy_score(y_test_PET, y_test_pred_PET)
conf_mat_train_cv_PET = confusion_matrix(y_train_PET, y_train_pred_cv_PET)
conf_mat_test_PET = confusion_matrix(y_test_PET, y_test_pred_PET)
sensitivity_train_PET = conf_mat_train_cv_PET[1, 1] / (conf_mat_train_cv_PET[1, 1] + conf_mat_train_cv_PET[1, 0])  # TP / (TP + FN)
sensitivity_test_PET = conf_mat_test_PET[1, 1] / (conf_mat_test_PET[1, 1] + conf_mat_test_PET[1, 0])  # TP / (TP + FN)
specificity_train_PET = conf_mat_train_cv_PET[0, 0] / (conf_mat_train_cv_PET[0, 0] + conf_mat_train_cv_PET[0, 1])
specificity_test_PET = conf_mat_test_PET[0, 0] / (conf_mat_test_PET[0, 0] + conf_mat_test_PET[0, 1])
ppv_train_PET = conf_mat_train_cv_PET[1, 1] / (conf_mat_train_cv_PET[1, 1] + conf_mat_train_cv_PET[0, 1])
npv_train_PET = conf_mat_train_cv_PET[0, 0] / (conf_mat_train_cv_PET[0, 0] + conf_mat_train_cv_PET[1, 0])
ppv_test_PET = conf_mat_test_PET[1, 1] / (conf_mat_test_PET[1, 1] + conf_mat_test_PET[0, 1])
npv_test_PET = conf_mat_test_PET[0, 0] / (conf_mat_test_PET[0, 0] + conf_mat_test_PET[1, 0])
train_precision_PET = precision_score(y_train_PET, y_train_pred_cv_PET)
test_precision_PET = precision_score(y_test_PET, y_test_pred_PET)
train_recall_PET = recall_score(y_train_PET, y_train_pred_cv_PET)
test_recall_PET = recall_score(y_test_PET, y_test_pred_PET)
train_f1_PET = f1_score(y_train_PET, y_train_pred_cv_PET)
test_f1_PET = f1_score(y_test_PET, y_test_pred_PET)
print(f"Sensitivity (Cross-Validation_PET): {sensitivity_train_PET:.4f}")
print(f"Specificity (Cross-Validation_PET): {specificity_train_PET:.4f}")
print(f"PPV (Cross-Validation_PET): {ppv_train_PET:.4f}")
print(f"NPV (Cross-Validation_PET): {npv_train_PET:.4f}")
print(f"Precision (Cross-Validation_PET): {train_precision_PET:.4f}")
print(f"Recall (Cross-Validation_PET): {train_recall_PET:.4f}")
print(f"F1 Score (Cross-Validation_PET): {train_f1_PET:.4f}")
print(f"Train Accuracy (Cross-Validation_PET): {train_accuracy_cv_PET:.4f}")
print(f"Sensitivity (Test_PET): {sensitivity_test_PET:.4f}")
print(f"Specificity (Test_PET): {specificity_test_PET:.4f}")
print(f"PPV (Test_PET): {ppv_test_PET:.4f}")
print(f"NPV (Test_PET): {npv_test_PET:.4f}")
print(f"Precision (Test_PET): {test_precision_PET:.4f}")
print(f"Recall (Test_PET): {test_recall_PET:.4f}")
print(f"F1 Score (Test_PET): {test_f1_PET:.4f}")
print(f"Test Accuracy_PET: {test_accuracy_PET:.4f}")

fpr_train_PET, tpr_train_PET, _ = roc_curve(y_train_PET, y_train_pred_proba_cv_PET)
fpr_test_PET, tpr_test_PET, _ = roc_curve(y_test_PET, y_test_pred_proba_PET)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train_PET = bootstrap_roc_auc(y_train_PET, y_train_pred_proba_cv_PET)
auc_train_mean_PET = np.mean(auc_scores_train_PET)
auc_train_std_PET = np.std(auc_scores_train_PET)
auc_train_lower_PET = norm.ppf(0.025, loc=auc_train_mean_PET, scale=auc_train_std_PET)
auc_train_upper_PET = norm.ppf(0.975, loc=auc_train_mean_PET, scale=auc_train_std_PET)
print(f"Cross-Validation AUC_PET: {auc_train_mean_PET:.4f} (95% CI: {auc_train_lower_PET:.4f}, {auc_train_upper_PET:.4f})")

auc_scores_test_PET = bootstrap_roc_auc(y_test_PET, y_test_pred_proba_PET)
auc_test_mean_PET = np.mean(auc_scores_test_PET)
auc_test_std_PET = np.std(auc_scores_test_PET)
auc_test_lower_PET = norm.ppf(0.025, loc=auc_test_mean_PET, scale=auc_test_std_PET)
auc_test_upper_PET = norm.ppf(0.975, loc=auc_test_mean_PET, scale=auc_test_std_PET)
print(f"Test AUC_PET: {auc_test_mean_PET:.4f} (95% CI: {auc_test_lower_PET:.4f}, {auc_test_upper_PET:.4f})")

Best parameters: {'C': 0.1, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}
Sensitivity (Cross-Validation_PET): 0.6979
Specificity (Cross-Validation_PET): 0.7745
PPV (Cross-Validation_PET): 0.7444
NPV (Cross-Validation_PET): 0.7315
Precision (Cross-Validation_PET): 0.7444
Recall (Cross-Validation_PET): 0.6979
F1 Score (Cross-Validation_PET): 0.7204
Train Accuracy (Cross-Validation_PET): 0.7374
Sensitivity (Test_PET): 0.7667
Specificity (Test_PET): 0.7875
PPV (Test_PET): 0.5750
NPV (Test_PET): 0.9000
Precision (Test_PET): 0.5750
Recall (Test_PET): 0.7667
F1 Score (Test_PET): 0.6571
Test Accuracy_PET: 0.7818
Cross-Validation AUC_PET: 0.7888 (95% CI: 0.7240, 0.8536)
Test AUC_PET: 0.8427 (95% CI: 0.7666, 0.9188)


In [6]:
### One example of dual-modality PET-CT stacking model. 
### Please adjust the code based on specific optimal feature selection and ML methods (refer to other single-modality model code format).
# Meta Classifier --- CatBoost
X_test_meta_features = np.column_stack((
    y_test_pred_proba_CT,  
    y_test_pred_proba_PET  
))

y_train_meta = y_train_CT
y_test_meta = y_test_CT

np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

X_train_meta_features = np.column_stack([
    y_train_pred_proba_cv_CT, 
    y_train_pred_proba_cv_PET   
])
X_test_meta_features = np.column_stack([
    y_test_pred_proba_CT,        
    y_test_pred_proba_PET       
])

np.random.seed(0)
class WrappedCatBoost(BaseEstimator, ClassifierMixin):
    def __init__(self, iterations=100, learning_rate=0.1, depth=6, l2_leaf_reg=1, border_count=1, random_seed=0):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.depth = depth
        self.l2_leaf_reg = l2_leaf_reg
        self.border_count = border_count
        self.random_seed = random_seed
        self.model_ = None

    def _prepare_params(self, params):
        return {**self.get_params(), **params}

    def fit(self, X, y, **fit_params):
        self.model_ = CatBoostClassifier(
            iterations=self.iterations,
            learning_rate=self.learning_rate,
            depth=self.depth,
            l2_leaf_reg=self.l2_leaf_reg,
            border_count=self.border_count,
            random_seed=self.random_seed,
            verbose=False
        )
        self.model_.fit(X, y, **fit_params)
        self.classes_ = np.unique(y)
        return self

    def predict(self, X):
        return self.model_.predict(X)

    def predict_proba(self, X):
        return self.model_.predict_proba(X)
    
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [3, 6, 10],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [16, 64, 128]
}
catboost = WrappedCatBoost()
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_meta_features, y_train_CT)
best_params = grid_search.best_params_
print("Best parameters for Logistic Regression Meta-learner:", best_params)

best_meta_model = grid_search.best_estimator_
y_train_meta_pred = cross_val_predict(best_meta_model, X_train_meta_features, y_train_CT, cv=5)
y_test_meta_pred = best_meta_model.predict(X_test_meta_features)
y_train_meta_pred_proba = cross_val_predict(best_meta_model, X_train_meta_features, y_train_CT, cv=5, method='predict_proba')[:, 1]
y_test_meta_pred_proba = best_meta_model.predict_proba(X_test_meta_features)[:, 1]

train_accuracy_meta = cross_val_score(best_meta_model, X_train_meta_features, y_train_CT, cv=5, scoring='accuracy').mean()
test_accuracy_meta = accuracy_score(y_test_CT, y_test_meta_pred)
train_precision_meta, train_recall_meta, train_f1_meta, _ = precision_recall_fscore_support(y_train_CT, y_train_meta_pred, average='binary')
test_precision_meta, test_recall_meta, test_f1_meta, _ = precision_recall_fscore_support(y_test_CT, y_test_meta_pred, average='binary')

def calculate_additional_metrics(y_true, y_pred, y_proba):
    conf_matrix = confusion_matrix(y_true, y_pred)
    sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])  # TP / (TP + FN)
    specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])  # TN / (TN + FP)
    ppv = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[0, 1])  # TP / (TP + FP)
    npv = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[1, 0])  # TN / (TN + FN)
    return sensitivity, specificity, ppv, npv, conf_matrix

sensitivity_train_meta, train_specificity, train_ppv, train_npv, conf_mat_train_meta = calculate_additional_metrics(y_train_CT, y_train_meta_pred, y_train_meta_pred_proba)
sensitivity_test_meta, test_specificity, test_ppv, test_npv, conf_mat_test_meta = calculate_additional_metrics(y_test_CT, y_test_meta_pred, y_test_meta_pred_proba)

print(f"Sensitivity (Cross-Validation Meta-learner): {sensitivity_train_meta:.4f}")
print(f"Specificity (Cross-Validation Meta-learner): {train_specificity:.4f}")
print(f"PPV (Cross-Validation Meta-learner): {train_ppv:.4f}")
print(f"NPV (Cross-Validation Meta-learner): {train_npv:.4f}")
print(f"Precision (Cross-Validation Meta-learner): {train_precision_meta:.4f}")
print(f"Recall (Cross-Validation Meta-learner): {train_recall_meta:.4f}")
print(f"F1 Score (Cross-Validation Meta-learner): {train_f1_meta:.4f}")
print(f"Train Accuracy (Cross-Validation  Meta-learner): {train_accuracy_meta:.4f}")
print(f"Sensitivity (Test Meta-learner): {sensitivity_test_meta:.4f}")
print(f"Specificity (Test Meta-learner): {test_specificity:.4f}")
print(f"PPV (Test Meta-learner): {test_ppv:.4f}")
print(f"NPV (Test Meta-learner): {test_npv:.4f}")
print(f"Precision (Test Meta-learner): {test_precision_meta:.4f}")
print(f"Recall (Test Meta-learner): {test_recall_meta:.4f}")
print(f"F1 Score (Test Meta-learner): {test_f1_meta:.4f}")
print(f"Test Accuracy (Test Meta-learner): {test_accuracy_meta:.4f}")
print("Confusion Matrix (Cross-Validation Meta-learner):\n", conf_mat_train_meta)
print("Confusion Matrix (Test Meta-learner):\n", conf_mat_test_meta)

np.random.seed(0)
fpr_train_meta, tpr_train_meta, _ = roc_curve(y_train_CT, y_train_meta_pred_proba)
fpr_test_meta, tpr_test_meta, _ = roc_curve(y_test_CT, y_test_meta_pred_proba)
auc_scores_train_meta = bootstrap_roc_auc(y_train_CT, y_train_meta_pred_proba)
auc_scores_test_meta = bootstrap_roc_auc(y_test_CT, y_test_meta_pred_proba)
auc_train_mean_meta = np.mean(auc_scores_train_meta)
auc_train_std_meta = np.std(auc_scores_train_meta)
auc_test_mean_meta = np.mean(auc_scores_test_meta)
auc_test_std_meta = np.std(auc_scores_test_meta)
auc_train_lower_meta = norm.ppf(0.025, loc=auc_train_mean_meta, scale=auc_train_std_meta)
auc_train_upper_meta = norm.ppf(0.975, loc=auc_train_mean_meta, scale=auc_train_std_meta)
auc_test_lower_meta = norm.ppf(0.025, loc=auc_test_mean_meta, scale=auc_test_std_meta)
auc_test_upper_meta = norm.ppf(0.975, loc=auc_test_mean_meta, scale=auc_test_std_meta)
print(f"Meta-learner Cross-Validation AUC: {auc_train_mean_meta:.4f} (95% CI: {auc_train_lower_meta:.4f}, {auc_train_upper_meta:.4f})")
print(f"Meta-learner Test AUC: {auc_test_mean_meta:.4f} (95% CI: {auc_test_lower_meta:.4f}, {auc_test_upper_meta:.4f})")

Best parameters for Logistic Regression Meta-learner: {'border_count': 16, 'depth': 6, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.05}
Sensitivity (Cross-Validation Meta-learner): 0.8854
Specificity (Cross-Validation Meta-learner): 0.7941
PPV (Cross-Validation Meta-learner): 0.8019
NPV (Cross-Validation Meta-learner): 0.8804
Precision (Cross-Validation Meta-learner): 0.8019
Recall (Cross-Validation Meta-learner): 0.8854
F1 Score (Cross-Validation Meta-learner): 0.8416
Train Accuracy (Cross-Validation  Meta-learner): 0.8392
Sensitivity (Test Meta-learner): 0.7333
Specificity (Test Meta-learner): 0.8500
PPV (Test Meta-learner): 0.6471
NPV (Test Meta-learner): 0.8947
Precision (Test Meta-learner): 0.6471
Recall (Test Meta-learner): 0.7333
F1 Score (Test Meta-learner): 0.6875
Test Accuracy (Test Meta-learner): 0.8182
Confusion Matrix (Cross-Validation Meta-learner):
 [[81 21]
 [11 85]]
Confusion Matrix (Test Meta-learner):
 [[68 12]
 [ 8 22]]
Meta-learner Cross-Validation AUC: 